In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords
from datetime import datetime
import webbrowser
import os

In [2]:
"""current_time = datetime.now().time()
start_time = datetime.strptime("10:00", "%H:%M").time()
end_time = datetime.strptime("17:00", "%H:%M").time()

if not (start_time <= current_time <= end_time):
    raise RuntimeError("Graph generation is allowed only between 10 AM and 5 PM.") """


'current_time = datetime.now().time()\nstart_time = datetime.strptime("10:00", "%H:%M").time()\nend_time = datetime.strptime("17:00", "%H:%M").time()\n\nif not (start_time <= current_time <= end_time):\n    raise RuntimeError("Graph generation is allowed only between 10 AM and 5 PM.") '

In [3]:
apps_df = pd.read_csv('Play Store Data.csv')

In [4]:
apps_df.head(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,07-Jan-18,1.0.0,4.0.3 and up


In [5]:
def convert_size(size):
    if 'M' in size:  # Convert MB to float
        return float(size.replace('M', ''))
    elif 'KB' in size:  # Convert KB to MB (1 MB = 1024 KB)
        return float(size.replace('KB', '')) / 1024
    else:  # Handle "Varies with device" or other invalid cases
        return None

In [6]:
apps_df['Size'] = apps_df['Size'].apply(lambda x: convert_size(x))

In [7]:
apps_df = apps_df.dropna(subset=['Size'])

In [8]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,"10,000+",Free,0,Everyone,Art & Design,07-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,"5,000,000+",Free,0,Everyone,Art & Design,01-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,"50,000,000+",Free,0,Teen,Art & Design,08-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10835,FR Forms,BUSINESS,NaN,0,9.6,10+,Free,0,Everyone,Business,29-Sep-16,1.1.5,4.0 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,"5,000+",Free,0,Everyone,Education,25-Jul-17,1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100+,Free,0,Everyone,Education,06-Jul-18,1,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5,"1,000+",Free,0,Everyone,Medical,20-Jan-17,1,2.2 and up


In [9]:
apps_df['Installs'] = apps_df['Installs'].str.replace('+', '', regex=False).str.replace(',', '', regex=False).astype(int)

In [10]:
apps_df = apps_df[apps_df['Rating']>4.0]
apps_df = apps_df[apps_df['Size']>10]

In [11]:
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], format='%d-%b-%y')

In [12]:
apps_df['Month'] = apps_df['Last Updated'].dt.month

In [13]:
apps_df = apps_df[apps_df['Month']==1]

In [14]:
apps_df=apps_df.dropna()

In [15]:
category_installs = apps_df.groupby('Category')['Installs'].sum().reset_index()

In [16]:
top_10_categories = category_installs.sort_values(by='Installs', ascending=False).head(10)

In [17]:
top_10_categories

,Category,Installs
7,FAMILY,177394820
19,SPORTS,120511000
10,GAME,103691000
5,ENTERTAINMENT,51000000
14,PERSONALIZATION,15060000
15,PHOTOGRAPHY,10000000
4,EDUCATION,2000000
17,SHOPPING,2000000
20,TOOLS,1010000
21,TRAVEL_AND_LOCAL,1001000
